In [2]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
from buffer import MultiModelActivationBuffer

from datasets import load_dataset
import torch as t

from nnsight import LanguageModel
from buffer import MultiModelActivationBuffer
from trainers.top_k import TopKTrainer, AutoEncoderTopK
from training import trainSAE
from einops import rearrange, einsum
import matplotlib.pyplot as plt
from tqdm import tqdm

device = "cuda:0"
dtype = t.bfloat16

In [3]:
layer = 4
expansion = 32
num_tokens = int(100e6)
out_batch_size = 2048

submodule_list = []
model_list = []
for step in [1, 16, 128, 512, 1000, 2000, 4000, 8000, 16000, 32000, 64000, 143000]:
    model = LanguageModel(
        "EleutherAI/pythia-70m", 
        revision=f"step{step}", 
        trust_remote_code=False, 
        device_map=device,
        torch_dtype=dtype,
        )
    for x in model.parameters():
        x.requires_grad = False
    model_list.append(model)
    submodule_list.append(model.gpt_neox.layers[layer])
    
activation_dim = 512
dictionary_size = expansion * activation_dim

dataset = load_dataset(
    'Skylion007/openwebtext', 
    split='train', 
    streaming=True,
    trust_remote_code=True
    )

class CustomData():
    def __init__(self, dataset):
        self.data = iter(dataset)

    def __iter__(self):
        return self

    def __next__(self):
        return next(self.data)['text']

data = CustomData(dataset)

buffer = MultiModelActivationBuffer(
    data=data,
    model_list=model_list,
    submodule_list=submodule_list,
    d_submodule=activation_dim, # output dimension of the model component
    n_ctxs=128,  # you can set this higher or lower dependong on your available memory
    device=device,
    refresh_batch_size=128,
    out_batch_size=out_batch_size,
    remove_bos=True
)  # buffer will yield batches of tensors of dimension = submodule's output dimension

The `GPTNeoXSdpaAttention` class is deprecated in favor of simply modifying the `config._attn_implementation`attribute of the `GPTNeoXAttention` class! It will be removed in v4.48


In [5]:
ae = AutoEncoderTopK.from_hf("jacobcd52/features_over_time", k=128, device=device)

/root/features_over_time/notebooks/../trainers/top_k.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = t.load(path)


In [4]:
def concat(x):
    return rearrange(x, 'b n d -> b (n d)')
def unconcat(x):
    return rearrange(x, 'b (n d) -> b n d', n=len(model_list))

In [ ]:
def get_feature_acts(buffer: MultiModelActivationBuffer, ae, n_batches):
    acts = []
    for _ in tqdm(range(n_batches)):
        x = buffer.get_seq_

        # whiten
        x = unconcat(x)
        x -= ae.act_mean
        x /= ae.act_std
        x = concat(x)

        # encode
        f = ae.encode(x)
        acts.append(f)
    return t.cat(acts)

In [10]:
f = get_feature_acts(buffer, ae, 1000)

  0%|          | 0/1000 [00:00<?, ?it/s]


AttributeError: 'MultiModelActivationBuffer' object has no attribute 'get_seq_batch'